# Quickstart notebook

In [0]:
from pyspark.sql.functions import *

## Geometry constructors and the Mosaic internal geometry format

Mosaic allows uses to create new Point geometries from a pair of Spark DoubleTypes columns.

In [0]:
from mosaic import st_point

lons = [-80., -80., -70., -70., -80.]
lats = [ 35.,  45.,  45.,  35.,  35.]

bounds_df = (
  spark
  .createDataFrame({"lon": lon, "lat": lat} for lon, lat in zip(lons, lats))
  .coalesce(1)
  .withColumn("point_geom", st_point("lon", "lat"))
)
bounds_df.show()

+----+-----+--------------------+
 lat| lon| point_geom|
+----+-----+--------------------+
35.0|-80.0|{1, [[[-80.0, 35....|
45.0|-80.0|{1, [[[-80.0, 45....|
45.0|-70.0|{1, [[[-70.0, 45....|
35.0|-70.0|{1, [[[-70.0, 35....|
35.0|-80.0|{1, [[[-80.0, 35....|
+----+-----+--------------------+

_Note: the first time you import from the Mosaic Python package, the supporting Scala classes will be copied to your cluster nodes. You can, therefore, expect this first command to take slightly longer to execute than subsequent actions._

Mosaic Point geometries can be aggregated into LineString and Polygon geometries using the respective constructors.

In [0]:
from mosaic import st_makeline

bounds_df = (
  bounds_df
  .groupBy()
  .agg(collect_list("point_geom").alias("bounding_coords"))
  .select(st_makeline("bounding_coords").alias("bounding_ring"))
)
bounds_df.show()

+--------------------+
 bounding_ring|
+--------------------+
{3, [[[-80.0, 35....|
+--------------------+

In [0]:
from mosaic import st_makepolygon

bounds_df = bounds_df.select(st_makepolygon("bounding_ring").alias("bounds"))
bounds_df.show()

+--------------------+
 bounds|
+--------------------+
{5, [[[-80.0, 35....|
+--------------------+

## Geometry clipping without an index

Mosaic implements set intersection functions: contains, intersects, overlaps etc. Here you can see `st_contains` being used to clip points by a polygon geometry.

In [0]:
tripsTable = spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")

In [0]:
from mosaic import st_contains
trips = (
  tripsTable
  .limit(5_000_000)
  .repartition(sc.defaultParallelism * 20)
  .drop("vendorId", "rateCodeId", "store_and_fwd_flag", "payment_type")
  .withColumn("pickup_geom", st_point("pickup_longitude", "pickup_latitude"))
  .withColumn("dropoff_geom", st_point("dropoff_longitude", "dropoff_latitude"))
  .crossJoin(bounds_df)
  .where(st_contains("bounds", "pickup_geom"))
  .where(st_contains("bounds", "dropoff_geom"))
  .cache()
)

In [0]:
trips.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
 CMT|2009-10-01 20:14:02|2009-10-01 20:25:12| 1| 1.0| -73.995029| 40.731688| null| -74.009906| 40.734639| 7.3| 0.0| null| 0.0| 0.0| 7.3|{1, [[[-73.995029...|{1, [[[-74.009906...|{5, [[[-80.0, 35....|
 VTS|2009-12-13 20:44:00|2009-12-13 20:49:00| 1| 0.86| -74.019097| 40.697303| null| -74.022585| 40.696682| 4.9| 0.5| 0.5| 0.0| 0.0| 5.9|{1, [[[-74.019097...|{1, [[[-74.022585...|{5, [[[-80.0, 35....|
 VTS|2009-02-25 03:09:00|2009-02-25 03:34:00| 3| 9.4| -74.003012| 40.733308| null| -73.879022| 40.756157| 23.7| 0.5| null| 0.0| 0.0| 24.2|{1, [[[-74.003012...|{1, [[[-73.879022...|{5, [[[-80.0, 35....|
 CMT|2009-01-26 20:17:32|2009-01-26 20:25:09| 1| 0.8| -73.998275| 40.725061| null| -73.988168| 40.721215| 6.6| 0.0| null| 0.0| 0.0| 6.6|{1, [[[-73.998275...|{1, [[[-73.988168...|{5, [[[-80.0, 35....|
 VTS|2009-10-29 15:23:00|2009-10-29 15:35:00| 5| 1.36| -73.990315| 40.734607| null| -73.996838| 40.747757| 7.7| 0.0| null| 2.0| 0.0| 9.7|{1, [[[-73.990315...|{1, [[[-73.996838...|{5, [[[-80.0, 35....|
 VTS|2010-06-02 12:57:00|2010-06-02 13:10:00| 1| 3.25| -74.01502| 40.713953| 1| -73.994892| 40.75023| 10.9| 0.0| 0.5| 0.0| 0.0| 11.4|{1, [[[-74.01502,...|{1, [[[-73.994892...|{5, [[[-80.0, 35....|
 VTS|2009-12-11 23:31:00|2009-12-11 23:34:00| 5| 0.51| -74.008582| 40.732705| null| -74.00919| 40.725968| 3.7| 0.5| 0.5| 0.0| 0.0| 4.7|{1, [[[-74.008582...|{1, [[[-74.00919,...|{5, [[[-80.0, 35....|
 VTS|2009-05-27 21:52:00|2009-05-27 21:56:00| 1| 0.63| -73.989672| 40.733835| null| -73.98474| 40.741957| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.989672...|{1, [[[-73.98474,...|{5, [[[-80.0, 35....|
 CMT|2009-05-26 15:16:42|2009-05-26 16:24:51| 1| 27.9| -73.991329| 40.697427| null| -73.781919| 40.648625| 60.9| 0.0| null| 0.0| 0.0| 60.9|{1, [[[-73.991329...|{1, [[[-73.781919...|{5, [[[-80.0, 35....|
 VTS|2009-09-25 12:08:00|2009-09-25 12:17:00| 1| 0.87| -73.989922| 40.733165| null| -73.993095| 40.742483| 6.5| 0.0| null| 1.0| 0.0| 7.5|{1, [[[-73.989922...|{1, [[[-73.993095...|{5, [[[-80.0, 35....|
 VTS|2009-12-08 18:48:00|2009-12-08 18:54:00| 5| 0.91| -74.00138| 40.719952| null| -73.994322| 40.713878| 5.3| 1.0| 0.5| 0.0| 0.0| 6.8|{1, [[[-74.00138,...|{1, [[[-73.994322...|{5, [[[-80.0, 35....|
 VTS|2009-11-02 19:31:00|2009-11-02 19:47:00| 5| 3.08| -73.993238| 40.72778| null| -73.991493| 40.76009| 10.9| 1.0| 0.5| 0.0| 0.0| 12.4|{1, [[[-73.993238...|{1, [[[-73.991493...|{5, [[[-80.0, 35....|
 DDS|2009-09-18 11:39:19|2009-09-18 11:54:20| 1| 0.3| -74.008426| 40.716548| null| -74.014158| 40.706377| 5.3| 0.0| null| 1.0| 0.0| 6.3|{1, [[[-74.008426...|{1, [[[-74.014158...|{5, [[[-80.0, 35....|
 VTS|2009-02-17 18:18:00|2009-02-17 18:38:00| 1| 2.75| -73.99006| 40.727742| null| -73.984877| 40.758912| 12.5| 1.0| null| 0.0| 0.0| 13.5|{1, [[[-73.99006,...|{1, [[[-73.984877...|{5, [[[-80.0, 35....|
 VTS|2009-05-22 09:49:00|2009-05-22 10:06:00| 5| 2.71| -73.990288| 40.732973| null| -73.960995| 40.75682| 10.9| 0.0| null| 2.0| 0.0| 12.9|{1, [[[-73.990288...|{1, [[[-73.960995...|{5, [[[-80.0, 35....|
 VTS|2009-05-07 18:25:00|2009-05-07 18:48:00| 1| 3.16| -74.002703| 40.72366| null| -73.972418| 40.757082| 12.9| 1.0| null| 0.0| 0.0| 13.9|{1, [[[-74.002703...|{1

## Read from GeoJson, compute some basic geometry attributes

You've seen how Mosaic can create geometries from Spark native data types but it also provides functions to translate Well Known Text (WKT), Well Known Binary (WKB) and GeoJSON representations to Mosaic geometries.

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("dbfs:/FileStore/shared_uploads/stuart.lynn@databricks.com/NYC_Taxi_Zones.geojson")
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

+-------------+-----------+--------+--------------------+--------------------+
 borough|location_id|objectid| zone| geometry|
+-------------+-----------+--------+--------------------+--------------------+
 EWR| 1| 1| Newark Airport|{6, [[[-74.184452...|
 Queens| 2| 2| Jamaica Bay|{6, [[[-73.823375...|
 Bronx| 3| 3|Allerton/Pelham G...|{6, [[[-73.847926...|
 Manhattan| 4| 4| Alphabet City|{6, [[[-73.971774...|
Staten Island| 5| 5| Arden Heights|{6, [[[-74.174217...|
Staten Island| 6| 6|Arrochar/Fort Wad...|{6, [[[-74.063673...|
 Queens| 7| 7| Astoria|{6, [[[-73.904136...|
 Queens| 8| 8| Astoria Park|{6, [[[-73.923340...|
 Queens| 9| 9| Auburndale|{6, [[[-73.785024...|
 Manhattan| 24| 24| Bloomingdale|{6, [[[-73.959536...|
 Queens| 10| 10| Baisley Park|{6, [[[-73.783266...|
 Brooklyn| 11| 11| Bath Beach|{6, [[[-74.001098...|
 Manhattan| 12| 12| Battery Park|{6, [[[-74.015657...|
 Manhattan| 13| 13| Battery Park City|{6, [[[-74.012441...|
 Bronx| 18| 18| Bedford Park|{6, [[[-73.885139...|
 Brooklyn| 25| 25| Boerum Hill|{6, [[[-73.981552...|
 Brooklyn| 14| 14| Bay Ridge|{6, [[[-74.034073...|
 Queens| 15| 15|Bay Terrace/Fort ...|{6, [[[-73.777403...|
 Brooklyn| 22| 22| Bensonhurst West|{6, [[[-73.992549...|
Staten Island| 23| 23|Bloomfield/Emerso...|{6, [[[-74.195686...|
+-------------+-----------+--------+--------------------+--------------------+
only showing top 20 rows

Mosaic provides a number of functions for extracting the properties of geometries. Here are some that are relevant to Polygon geometries:

In [0]:
from mosaic import st_area, st_length
(
  geoJsonDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_length("geometry"))
  .select("geometry", "calculatedArea", "calculatedLength")
).show()

+--------------------+--------------------+--------------------+
 geometry| calculatedArea| calculatedLength|
+--------------------+--------------------+--------------------+
{6, [[[-74.184452...|7.823067885002562E-4| 0.1163574531886787|
{6, [[[-73.823375...|0.001422779097814599| 0.8431218810128789|
{6, [[[-73.847926...|3.144141568206508E-4| 0.08434110590105784|
{6, [[[-73.971774...| 7.94539194214528E-5| 0.03559982116907461|
{6, [[[-74.174217...|4.979574893632193E-4| 0.09214648985735088|
{6, [[[-74.063673...|4.046076598541714E-4| 0.12905753305870837|
{6, [[[-73.904136...|3.897879892739018...| 0.10741717112348065|
{6, [[[-73.923340...|2.658771690499071...|0.027590691195158792|
{6, [[[-73.785024...|3.384438031986125...| 0.0997840924710119|
{6, [[[-73.959536...|4.193691000471793E-5|0.034022318280805826|
{6, [[[-73.783266...|4.358238180810105...| 0.09983947941552027|
{6, [[[-74.001098...|2.029913532381591E-4| 0.07243934539937844|
{6, [[[-74.015657...|1.078595394871048...| 0.01759380786230351|
{6, [[[-74.012441...|4.874002713725957E-5| 0.04726120195723124|
{6, [[[-73.885139...|1.488501639473639E-4| 0.06979954985700013|
{6, [[[-73.981552...|1.241682673558662...| 0.04714581993191898|
{6, [[[-74.034073...|6.636590898190788E-4| 0.13622530962992194|
{6, [[[-73.777403...|4.588438269978550...| 0.1295148763926726|
{6, [[[-73.992549...|4.613649657988681...| 0.11844001178496774|
{6, [[[-74.195686...|0.002090822778761513| 0.3007595308122533|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
geoJsonDF.count()

Out[29]: 263

## Example point-in-poly with indexing

Mosaic has built-in support for the popular spatial indexing library, H3. The user has access to functions for generating point indices and the sets of indices covering polygons, allowing point-in-polygon joins to be transformed into deterministic SQL joins.

In [0]:
from mosaic import point_index

trips_with_geom = (
  trips
  .withColumn("pickup_h3", point_index(lng="pickup_longitude", lat="pickup_latitude", resolution=lit(10)))
  .withColumn("dropoff_h3", point_index(lng="dropoff_longitude", lat="dropoff_latitude", resolution=lit(10)))
)

trips_with_geom.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
 CMT|2009-10-01 20:14:02|2009-10-01 20:25:12| 1| 1.0| -73.995029| 40.731688| null| -74.009906| 40.734639| 7.3| 0.0| null| 0.0| 0.0| 7.3|{1, [[[-73.995029...|{1, [[[-74.009906...|{5, [[[-80.0, 35....|622236750715715583|622236750524612607|
 VTS|2009-12-13 20:44:00|2009-12-13 20:49:00| 1| 0.86| -74.019097| 40.697303| null| -74.022585| 40.696682| 4.9| 0.5| 0.5| 0.0| 0.0| 5.9|{1, [[[-74.019097...|{1, [[[-74.022585...|{5, [[[-80.0, 35....|622236750646149119|622236750645264383|
 VTS|2009-02-25 03:09:00|2009-02-25 03:34:00| 3| 9.4| -74.003012| 40.733308| null| -73.879022| 40.756157| 23.7| 0.5| null| 0.0| 0.0| 24.2|{1, [[[-74.003012...|{1, [[[-73.879022...|{5, [[[-80.0, 35....|622236750719352831|622236723948191743|
 CMT|2009-01-26 20:17:32|2009-01-26 20:25:09| 1| 0.8| -73.998275| 40.725061| null| -73.988168| 40.721215| 6.6| 0.0| null| 0.0| 0.0| 6.6|{1, [[[-73.998275...|{1, [[[-73.988168...|{5, [[[-80.0, 35....|622236750706835455|622236750717485055|
 VTS|2009-10-29 15:23:00|2009-10-29 15:35:00| 5| 1.36| -73.990315| 40.734607| null| -73.996838| 40.747757| 7.7| 0.0| null| 2.0| 0.0| 9.7|{1, [[[-73.990315...|{1, [[[-73.996838...|{5, [[[-80.0, 35....|622236723428556799|622236723431309311|
 VTS|2010-06-02 12:57:00|2010-06-02 13:10:00| 1| 3.25| -74.01502| 40.713953| 1| -73.994892| 40.75023| 10.9| 0.0| 0.5| 0.0| 0.0| 11.4|{1, [[[-74.01502,...|{1, [[[-73.994892...|{5, [[[-80.0, 35....|622236750652309503|622236723439173631|
 VTS|2009-12-11 23:31:00|2009-12-11 23:34:00| 5| 0.51| -74.008582| 40.732705| null| -74.00919| 40.725968| 3.7| 0.5| 0.5| 0.0| 0.0| 4.7|{1, [[[-74.008582...|{1, [[[-74.00919,...|{5, [[[-80.0, 35....|622236750524645375|622236750711193599|
 VTS|2009-05-27 21:52:00|2009-05-27 21:56:00| 1| 0.63| -73.989672| 40.733835| null| -73.98474| 40.741957| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.989672...|{1, [[[-73.98474,...|{5, [[[-80.0, 35....|622236723428491263|622236723427246079|
 CMT|2009-05-26 15:16:42|2009-05-26 16:24:51| 1| 27.9| -73.991329| 40.697427| null| -73.781919| 40.648625| 60.9| 0.0| null| 0.0| 0.0| 60.9|{1, [[[-73.991329...|{1, [[[-73.781919...|{5, [[[-80.0, 35....|622236750729084927|622236735751716863|
 VTS|2009-09-25 12:08:00|2009-09-25 12:17:00| 1| 0.87| -73.989922| 40.733165| null| -73.993095| 40.742483| 6.5| 0.0| null| 1.0| 0.0| 7.5|{1, [[[-73.989922...|{1, [[[-73.993095...|{5, [[[-80.0, 35....|622236723428589567|622236723426459647|
 VTS|2009-12-08 18:48:00|2009-12-08 18:54:00| 5| 0.91| -74.00138| 40.719952| null| -73.994322| 40.713878| 5.3| 1.0| 0.5| 0.0| 0.0| 6.8|{1, [[[-74.00138,...|{1, [[[-73.994322...|{5, [[[-80.0, 35....|622236750705983487|622236750709063679|
 VTS|2009-11-02 19:31:00|2009-11-02 19:47:00| 5| 3.08| -73.993238| 40.72778| null| -73.991493| 40.76009| 10.9| 1.0| 0.5| 0.0| 0.0| 12.4|{1, [[[-73.993238...|{1, [[[-73.991493...|{5, [[[-80.0, 35....|622236750714044415|622236750599815167|
 DDS|2009-09-18 11:39:19|2009-09-18 11:54:20| 1| 0.3| -74.008426| 40.716548| null| -74.014158| 40.706377| 5.3| 0.0| null| 1.0| 0.0| 6.3|{1, [[[-74.008426...|{1, [[[-74.014158...|{5, [[[-80.0, 35....|62223675071

In [0]:
from mosaic import polyfill

neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", explode(polyfill("geometry", lit(10))).alias("h3"))
  .drop("geometry")
)

neighbourhoods.show()

+-------+-----------+--------+--------------------+------------------+
borough|location_id|objectid| zone| h3|
+-------+-----------+--------+--------------------+------------------+
 Queens| 207| 207|Saint Michaels Ce...|622236723991183359|
 Queens| 207| 207|Saint Michaels Ce...|622236723989315583|
 Queens| 207| 207|Saint Michaels Ce...|622236723950878719|
 Queens| 207| 207|Saint Michaels Ce...|622236723988201471|
 Queens| 207| 207|Saint Michaels Ce...|622236723991216127|
 Queens| 207| 207|Saint Michaels Ce...|622236723991248895|
 Queens| 207| 207|Saint Michaels Ce...|622236723990134783|
 Queens| 207| 207|Saint Michaels Ce...|622236723990167551|
 Queens| 207| 207|Saint Michaels Ce...|622236723950223359|
 Queens| 207| 207|Saint Michaels Ce...|622236723988299775|
 Queens| 207| 207|Saint Michaels Ce...|622236723950256127|
 Queens| 207| 207|Saint Michaels Ce...|622236723951009791|
 Queens| 207| 207|Saint Michaels Ce...|622236723988332543|
 Queens| 207| 207|Saint Michaels Ce...|622236723951042559|
 Queens| 207| 207|Saint Michaels Ce...|622236723991379967|
 Queens| 207| 207|Saint Michaels Ce...|622236723950682111|
 Queens| 207| 207|Saint Michaels Ce...|622236723950714879|
 Queens| 207| 207|Saint Michaels Ce...|622236723991052287|
 Queens| 207| 207|Saint Michaels Ce...|622236723950747647|
 Queens| 207| 207|Saint Michaels Ce...|622236723991085055|
+-------+-----------+--------+--------------------+------------------+
only showing top 20 rows

In [0]:
joined_df = trips_with_geom.alias("t").join(neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
joined_df.count()

Out[32]: 4934121

## Mosaic spatial join optimizations

Mosaic provides easy access to the optimized spatial join technique described in [this](https://databricks.com/blog/2021/10/11/efficient-point-in-polygon-joins-via-pyspark-and-bng-geospatial-indexing.html) blog post.

(this step is required due to incompatibility between Databricks Runtime and open source Spark)

In [0]:
%run ../GeneratorsHotfix

In [0]:
from mosaic.patch import mosaic_explode

mosaic_neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", mosaic_explode("geometry", lit(10)))
  .drop("geometry")
)

mosaic_neighbourhoods.show()

+-------+-----------+--------+--------------------+-------+------------------+--------------------+
borough|location_id|objectid| zone|is_core| h3| wkb|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950682111| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950845951| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723951009791| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988332543| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950780415| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991085055| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991248895| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950714879| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991183359| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988299775| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950747647| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991052287| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991216127| null|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723991379967|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723951140863|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723990167551|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723991117823|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950288895|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723988365311|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950878719|[01 03 00 00 00 0...|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
only showing top 20 rows

Now the two datasets can be joined first on H3 index, with any false positives removed through a contains filter on a much simpler geometry.

In [0]:
mosaic_joined_df = (
  trips_with_geom.alias("t")
  .join(mosaic_neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
  .where(
    ~col("is_core") | 
    st_contains("wkb", "pickup_geom")
  )
)

mosaic_joined_df.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3| borough|location_id|objectid| zone|is_core| h3| wkb|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
 CMT|2009-01-26 20:17:32|2009-01-26 20:25:09| 1| 0.8| -73.998275| 40.725061| null| -73.988168| 40.721215| 6.6| 0.0| null| 0.0| 0.0| 6.6|{1, [[[-73.998275...|{1, [[[-73.988168...|{5, [[[-80.0, 35....|622236750706835455|622236750717485055|Manhattan| 211| 211| SoHo| false|622236750706835455|[01 03 00 00 00 0...|
 CMT|2009-01-26 20:17:32|2009-01-26 20:25:09| 1| 0.8| -73.998275| 40.725061| null| -73.988168| 40.721215| 6.6| 0.0| null| 0.0| 0.0| 6.6|{1, [[[-73.998275...|{1, [[[-73.988168...|{5, [[[-80.0, 35....|622236750706835455|622236750717485055|Manhattan| 114| 114|Greenwich Village...| false|622236750706835455|[01 03 00 00 00 0...|
 VTS|2009-10-29 15:23:00|2009-10-29 15:35:00| 5| 1.36| -73.990315| 40.734607| null| -73.996838| 40.747757| 7.7| 0.0| null| 2.0| 0.0| 9.7|{1, [[[-73.990315...|{1, [[[-73.996838...|{5, [[[-80.0, 35....|622236723428556799|622236723431309311|Manhattan| 234| 234| Union Sq| false|622236723428556799|[01 03 00 00 00 0...|
 VTS|2009-10-29 15:23:00|2009-10-29 15:35:00| 5| 1.36| -73.990315| 40.734607| null| -73.996838| 40.747757| 7.7| 0.0| null| 2.0| 0.0| 9.7|{1, [[[-73.990315...|{1, [[[-73.996838...|{5, [[[-80.0, 35....|622236723428556799|622236723431309311|Manhattan| 113| 113|Greenwich Village...| false|622236723428556799|[01 03 00 00 00 0...|
 VTS|2009-05-27 21:52:00|2009-05-27 21:56:00| 1| 0.63| -73.989672| 40.733835| null| -73.98474| 40.741957| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.989672...|{1, [[[-73.98474,...|{5, [[[-80.0, 35....|622236723428491263|622236723427246079|Manhattan| 234| 234| Union Sq| false|622236723428491263|[01 03 00 00 00 0...|
 VTS|2009-05-27 21:52:00|2009-05-27 21:56:00| 1| 0.63| -73.989672| 40.733835| null| -73.98474| 40.741957| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.989672...|{1, [[[-73.98474,...|{5, [[[-80.0, 35....|622236723428491263|622236723427246079|Manhattan| 113| 113|Greenwich Village...| false|622236723428491263|[01 03 00 00 00 0...|
 VTS|2009-05-27 21:52:00|2009-05-27 21:56:00| 1| 0.63| -73.989672| 40.733835| null| -73.98474| 40.741957| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.989672...|{1, [[[-73.98474,...|{5, [[[-80.0, 35....|622236723428491263|622236723427246079|Manhattan| 107| 107| Gramercy| false|622236723428491263|[01 03 00 00 00 0...|
 VTS|2009-05-27 21:52:00|2009-05-27 21:56:00| 1| 0.63| -73.989672| 40.733835| null| -73.98474| 40.741957| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.989672...|{1, [[[-73.98474,...|{5, [[[-80.0, 35....|622236723428491263|622236723427246079|Manhattan| 79| 79| East Village| false|622236723428491263|[01 03 00 00 00 0...|
 CMT|2009-05-26 15:16:42|2009-05-26 16:24:51| 1| 27.9| -73.991329| 40.697427| null| -73.781919| 40.648625| 60.9| 0.0| null| 0.0| 0.0| 60.9|{1, [[[-73.991329...|{1, [[[-73.781919...|{5, [[[-80.0, 35....|622236750729084927|622236735751716863| Brooklyn| 65| 65|Down